In [90]:
# Import 
import pandas as pd
import numpy as np
from scipy import signal
import openpyxl

In [91]:
# Input file
folder_path = r'D:\iriwa\デスクトップ\Code\Project_Python\Research'
file_name = r'\yoshiyama.txt'

output_filename = '吉山'+'(EEG)'


input_data = pd.read_table(folder_path + file_name, header=None, skiprows=6, names=('time','O1','O2'))
input_data = input_data.interpolate()
print(input_data)

time        O1        O2
0           0.000 -0.081250 -0.306250
1           0.001 -0.156250 -0.028125
2           0.002  0.093750  0.181250
3           0.003  0.362500  0.318750
4           0.004  0.653125  0.418750
...           ...       ...       ...
1846495  1846.495  0.506250 -0.984375
1846496  1846.496  0.393750 -1.250000
1846497  1846.497 -0.221875 -1.237500
1846498  1846.498 -0.646875 -1.009375
1846499  1846.499 -0.721875 -0.731250

[1846500 rows x 3 columns]


In [92]:
N  = 1024   # Sampling size # サンプリングサイズ(FFTサイズ)
dt = 0.001  # Sampling period[s]        # サンプリング周期[s]
fs = 1/dt   # Sampling rate[Hz]         # サンプリング周波数[Hz]
TIME = int(len(input_data)/fs)  # Data time[s]  #データ時間[s]
print('%d秒分のデータ'%TIME)
time = pd.DataFrame({'time': [i for i in range(TIME)]})
print(time)

1846秒分のデータ
      time
0        0
1        1
2        2
3        3
4        4
...    ...
1841  1841
1842  1842
1843  1843
1844  1844
1845  1845

[1846 rows x 1 columns]


In [93]:
filter1 = signal.firwin(numtaps=16, cutoff=[1,30],pass_zero=False, fs=1/dt) # Filter (bandpass)
wf = signal.hamming(N)  # Window function(hamming)

In [94]:
output_data = []
for n in range(2):
    filter_data = signal.lfilter(filter1, 1, input_data.iloc[:, n+1])   # Filtering(bandpass)   # フィルタリング(バンドパス)
    # cut_data    = []    #cutting for FFT   # FFT用に整形されたデータ
    # window_data = []    #apply window function # 窓関数適用データ
    
    MPF   = [np.nan for i in range(TIME)]
    alpha = [np.nan for i in range(TIME)]
    beta  = [np.nan for i in range(TIME)]
    theta = [np.nan for i in range(TIME)]
    delta = [np.nan for i in range(TIME)]

    for i in range(TIME):
        # Formatted to 1s(1000+24data) & Apply window function  # 1秒(1000+24個)を切り取り & 窓関数適用
        Format_data = [filter_data[j+i*int(fs)] * wf[j] for j in range(N)]  # Formatted to 1s(1000+24data)  # 1秒(1000+24個)を切り取り

        #Func_FFT() #FFT    #高速フーリエ変換
        F = np.fft.fft(Format_data)
        amplitude = np.abs(F)   # Amplitude     # 振幅スペクトル(F/N/2)
        power = amplitude ** 2        # Power         # パワースペクトル
        density = power/(fs/N)     # Power Density # パワースペクトル密度
        freq = np.fft.fftfreq(N, d=dt)
        f_l = int(len(freq)/2)
        
        #Func_cal
        sum_alpha = 0
        sum_beta = 0
        sum_theta = 0
        sum_delta = 0
        total_fp = 0
        total_power = 0

        for l in range(f_l):
            if freq[l] >= 1.0 and freq[l] <= 30: 
                total_power += amplitude[l]
                total_fp = total_fp + (amplitude[l] * freq[l])
                if freq[l] >= 1.0  and freq[l] <= 4.0:
                    sum_delta += amplitude[l]
                if freq[l] >= 4.0  and freq[l] <= 8.0:
                    sum_theta += amplitude[l]
                if freq[l] >= 8.0  and freq[l] <= 13.0:
                    sum_alpha += amplitude[l]
                if freq[l] >= 13.0 and freq[l] <= 30.0:
                    sum_beta  += amplitude[l]

        MPF[i]   = total_fp  / total_power
        alpha[i] = sum_alpha / total_power 
        beta[i]  = sum_beta  / total_power
        theta[i] = sum_theta / total_power
        delta[i] = sum_delta / total_power
        
    output_data.append(MPF)
    output_data.append(alpha)
    output_data.append(beta)
    output_data.append(theta)
    output_data.append(delta)
df = pd.DataFrame(output_data, index=['O1_MPF', 'O1_alpha', 'O1_beta', 'O1_theta', 'O1_delta',\
                                            'O2_MPF', 'O2_alpha', 'O2_beta', 'O2_theta', 'O2_delta']).T
print(df)

O1_MPF  O1_alpha   O1_beta  O1_theta  O1_delta    O2_MPF  O2_alpha  \
0     13.686829  0.319327  0.448741  0.114465  0.117466  8.129601  0.131708   
1     13.281665  0.275267  0.418969  0.114246  0.191518  9.042647  0.144431   
2     14.006504  0.197290  0.533035  0.186214  0.083460  9.191498  0.142544   
3      9.822893  0.136853  0.316846  0.162246  0.384055  3.406144  0.028116   
4     12.621599  0.306302  0.444937  0.122499  0.126262  5.281067  0.071322   
...         ...       ...       ...       ...       ...       ...       ...   
1841   6.707479  0.122120  0.111139  0.272198  0.494543  7.020728  0.112570   
1842   7.227837  0.147290  0.140346  0.340311  0.372053  4.905181  0.039817   
1843   7.224495  0.173272  0.144755  0.240751  0.441222  4.831436  0.048145   
1844   7.679220  0.186126  0.140424  0.358487  0.314964  5.182000  0.076747   
1845   7.543445  0.124626  0.147187  0.347134  0.381053  3.547394  0.027317   

       O2_beta  O2_theta  O2_delta  
0     0.195265  0.28431

In [95]:
output = pd.concat([time, df], axis=1)
print(output)

time     O1_MPF  O1_alpha   O1_beta  O1_theta  O1_delta    O2_MPF  \
0        0  13.686829  0.319327  0.448741  0.114465  0.117466  8.129601   
1        1  13.281665  0.275267  0.418969  0.114246  0.191518  9.042647   
2        2  14.006504  0.197290  0.533035  0.186214  0.083460  9.191498   
3        3   9.822893  0.136853  0.316846  0.162246  0.384055  3.406144   
4        4  12.621599  0.306302  0.444937  0.122499  0.126262  5.281067   
...    ...        ...       ...       ...       ...       ...       ...   
1841  1841   6.707479  0.122120  0.111139  0.272198  0.494543  7.020728   
1842  1842   7.227837  0.147290  0.140346  0.340311  0.372053  4.905181   
1843  1843   7.224495  0.173272  0.144755  0.240751  0.441222  4.831436   
1844  1844   7.679220  0.186126  0.140424  0.358487  0.314964  5.182000   
1845  1845   7.543445  0.124626  0.147187  0.347134  0.381053  3.547394   

      O2_alpha   O2_beta  O2_theta  O2_delta  
0     0.131708  0.195265  0.284319  0.388708  
1     0.144

In [96]:
# Output data
output_folder_path = 'D:\\iriwa\\デスクトップ\\Code\\Project_Python\\Research\\EEG'
output.to_excel(output_folder_path + '\\' + output_filename +'.xlsx', sheet_name=output_filename, index=False)
